# Knapsack

## Model Formulation

### Sets and Indices

$i \in I$: Index and set of items.

### Parameters

$v_{i} \in \mathbb{R}^+$: Value of item $i \in I$.

$w_{i} \in \mathbb{R}^+$: Weight of item $i \in I$.

$K \in \mathbb{R}^+$: Capacity of the knapsack

### Decision Variables

$x_{i} \in \{0, 1 \}$: This variable is equal to 1 if we take item $i \in I$; and 0 otherwise.

### Objective Function

- **Total value**. We want to maximize the total value of the items selected to go into the knapsack. This is the sum of the values of the selected items.

\begin{equation}
\max Z= \displaystyle \sum_{i=0}^{n-1} v_ix_i
\tag{0}
\end{equation}

### Constraints

- **Capacity**. The total weight of the selected items $i \in I$ must not exceed the capacity $K$ of the knapsack:

\begin{equation}
\displaystyle \sum_{i=0}^{n-1} w_ix_i \leq K \text{ where } x_i \in \{0,1\} \ \forall i \in \{ 0, \ldots, n-1 \}
\tag{1}
\end{equation}

# Python implementation

## Import the libraries

The following code imports the required libraries.

In [6]:
from collections import namedtuple
import pandas as pd

ModuleNotFoundError: No module named 'ortools'

## Item definition

The item includes its index, value and weight.

In [ ]:
Item = namedtuple("Item", ['index', 'value', 'weight'])

## Create the data

The code below creates the data for the problem.  

### Read the file

In [ ]:
url = 'https://raw.githubusercontent.com/jacubero/Optimization/main/knapsack/data/ks_30_0'
df = pd.read_csv(url, sep=" ", header=None)
df.head()

In [ ]:
item_count = int(df.at[0,0])
capacity = int(df.at[0,1])

print("Number of items =", item_count)
print("Capacity of the knapsack =", capacity)

items = []

for i in range(1, item_count+1):
    items.append(Item(i,df.at[i,0],df.at[i,1]))

## Get expectation

Get the max value expectation from current capacity and current undecided item.

In [ ]:
def get_expectation(items, capacity, start):
	expectation = 0.0
	for i in range(start, len(items)):
		item = items[i]
		if capacity >= item.weight:
			expectation += item.value
			capacity -= item.weight
		
		# if current capacity is not enough to carry the whole item, then put a fraction of it into the knapsack
		# and add the same fraction of its value to the expectation
		else:
			expectation += item.value * capacity / item.weight
			break

	return expectation

### Search

Find max value and the take/no-take choice for each item.

In [ ]:
def search(items,  capacity):
	
	max_value = 0.0
	max_taken = [0]*len(items)

	# To prevent from stack-overflow, instead of using plain recursion here I maintain the stack myself
	# a stack element includes 5 parts:
	# value:         value accumulated so far
	# capacity:      left capacity
	# expectation:   upper bound of value that can get with the left capacity
	# taken:         current take/no-take choice of each item
	# pos:           next item to consider

	start_value = 0.0
	start_capacity = capacity
	start_expectation = get_expectation(items, capacity, 0)
	list_size = len(items)
	start_taken = [0]*list_size
	start_pos = 0

	StackElem = namedtuple("StackElem", ['value', 'capacity', 'expectation', 'taken', 'pos'])
	stack = []
	stack.append(StackElem(start_value, start_capacity, start_expectation, start_taken, start_pos))

	while stack:
		Current = stack.pop()

		# If left capacity is not enough, then backtrack
		if Current.capacity < 0:
			continue
		
		# If current expectation is smaller than the best value, then backtrack
		if Current.expectation <= max_value:
			continue

		# If max value is smaller than current value, update max value and its item-take choices
		if max_value < Current.value:
			max_value = Current.value
			max_taken = Current.taken

		# If next item to consider dose not exist, then backtrack
		if Current.pos >= list_size: 
			continue

		CurrentItem = items[Current.pos]
    
		# Try not to take the next item
		notake_value = Current.value
		notake_capacity = Current.capacity
		notake_expectation = notake_value + get_expectation(items, notake_capacity, Current.pos + 1)
		notake_taken = Current.taken
        
		stack.append(StackElem(notake_value, notake_capacity, notake_expectation, notake_taken, Current.pos + 1))
    
		# Try to take the next item
		take_value = Current.value + CurrentItem.value
		take_capacity = Current.capacity - CurrentItem.weight
		take_expectation = take_value + get_expectation(items, take_capacity, Current.pos + 1)
		take_taken = Current.taken
		take_taken[CurrentItem.index-1] = 1
        
		stack.append(StackElem(take_value, take_capacity, take_expectation, take_taken, Current.pos + 1))
	
	return max_value, max_taken

### Sort the items

Sort the items, so that they are in value density decreasing order

In [ ]:
sorted(items, key=lambda x: getattr(x, 'value')/getattr(x, 'weight'), reverse=True)

### Search

In [ ]:
value, taken = search(items, capacity)

## Prints the solution

Prints the solution in the specified output format

In [ ]:
output_data = str(value) + ' ' + str(1) + '\n'
output_data += ' '.join(map(str, taken))

print(output_data)

# Shell script execution

## Upload data

In [ ]:
%%shell

wget -nc -P ./data https://raw.githubusercontent.com/jacubero/Optimization/main/knapsack/data/data.zip
cd data
unzip data.zip
rm data.zip
ls

## Execute solver

In [ ]:
%%shell

wget -nc https://raw.githubusercontent.com/jacubero/Optimization/main/knapsack/or_branch_and_bound/solver.py

mkdir -p or_branch_and_bound

for file in $(ls ./data/*)
do
  filename="$(basename "$file")"
  python3 solver.py $file > ./or_branch_and_bound/$filename.orb
done